In [1]:
import pandas as pd
import numpy as np

In [11]:
data = pd.read_csv('.\datasets\hfi_cc_2018.csv')
data = data[[
'year',
'countries',
'region',
'pf_rank',
'pf_score',
#'pf_rol_civil',
'pf_ss_homicide',
'pf_ss_disappearances',
'pf_movement',
'pf_religion_estop',
#'pf_association_political_establish',
'pf_association',
'pf_expression_internet',
'pf_identity_divorce',
'ef_government_tax',
'ef_government',
'ef_money_currency',
'ef_trade_tariffs_mean',
'ef_trade_movement_visit']]
before_2016_data = data[data['year'] == 2016]
before_2016_data['year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)

eemaldatavad_riigid = data.loc[(data["pf_rank"].isnull()) & (data["year"] == 2010)]["countries"].tolist() #Valime välja riigid millel puudub varasem info

eemaldatavad_aastad = [2008, 2009]
eemaldatavad_riigid = set(eemaldatavad_riigid) #Eemaldab kordused

for riik in eemaldatavad_riigid:
    data = data[~data["countries"].isin(eemaldatavad_riigid)] #Eemaldame riigid

for aasta in eemaldatavad_aastad:
    data = data[~data["year"].isin(eemaldatavad_aastad)] #Eemaldame aastad mille kohta meil on vähem infot

olemasolevad_riigid = set(data["countries"].tolist())

,year,countries,region,pf_rank,pf_score,pf_ss_homicide,pf_ss_disappearances,pf_movement,pf_religion_estop,pf_association,pf_expression_internet,pf_identity_divorce,ef_government_tax,ef_government,ef_money_currency,ef_trade_tariffs_mean,ef_trade_movement_visit
0,2016,Albania,Eastern Europe,57.0,7.596281,8.920429,10.000000,6.666667,10.000000,10.000000,10.000000,5.0,8.0,7.935564,10.0,9.24,8.296923
1,2016,Algeria,Middle East & North Africa,147.0,5.281772,9.456254,8.842060,5.000000,5.000000,5.000000,7.500000,0.0,4.5,3.616782,5.0,6.22,1.106256
2,2016,Angola,Sub-Saharan Africa,117.0,6.111324,8.060260,8.500000,5.000000,10.000000,4.000000,7.500000,10.0,9.5,6.496685,5.0,7.72,0.110626
3,2016,Argentina,Latin America & the Caribbean,42.0,8.099696,7.622974,9.498175,10.000000,7.500000,7.500000,10.000000,10.0,4.0,5.346056,10.0,7.26,7.965046
4,2016,Armenia,Caucasus & Central Asia,84.0,6.912804,8.808750,9.349563,6.666667,5.000000,6.500000,7.500000,5.0,5.0,7.003310,10.0,8.76,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,2010,Venezuela,Latin America & the Caribbean,115.0,6.353238,0.000000,8.000000,10.000000,7.083333,5.583333,6.666667,NaN,7.0,4.958166,5.0,7.50,6.526913
1130,2010,Vietnam,South Asia,99.0,6.679871,9.395971,10.000000,5.000000,4.166667,2.750000,3.333333,NaN,5.0,7.038235,0.0,8.04,0.553128
1131,2010,"Yemen, Rep.",Middle East & North Africa,151.0,4.730448,8.137823,5.599981,3.333333,5.833333,5.583333,3.333333,NaN,NaN,6.603117,10.0,NaN,0.221251
1132,2010,Zambia,Sub-Saharan Africa,122.0,6.087383,7.649103,10.000000,8.333333,5.833333,4.250000,10.000000,NaN,8.0,8.340892,10.0,7.32,3.650646


#### Kood millega lisasin GDP teise datasetti
data_2 = pd.read_csv('.\datasets\Dataset2.csv')
gdp_data = pd.read_csv('.\datasets\gdp_data.csv')

gdp_data = gdp_data.drop(columns=['Country Code', 'Series Name', 'Series Code', '1990 [YR1990]', '2000 [YR2000]', '2019 [YR2019]', '2017 [YR2017]', '2018 [YR2018]'])

for year in range(2010, 2017, 1):
    print(year)
    gdp_data.rename(columns={(str(year) + " [YR" + str(year) + "]"): year}, inplace=True)

gdp_data = gdp_data.melt(id_vars=["Country Name"], 
        var_name="Year", 
        value_name="GDP")

#gdp_data = gdp_data.loc[gdp_data['Country Name'] in data_2['Country Name']]

gdp_data = gdp_data.reset_index()
for i in range(len(gdp_data)):
    if (gdp_data.loc[i]['Country Name']) not in set(data_2['Country Name']):
        gdp_data = gdp_data.drop(i, axis=0)




data_2 = data_2.merge(gdp_data, on=['Country Name', 'Year'], how='inner').drop(columns=['index'])
#data_2.to_csv("dataset2.csv", index=False)

In [10]:
data_2 = pd.read_csv('.\datasets\dataset2.csv')

before_2016_data = data_2[data_2['Year'] == 2008]
before_2016_data['Year'].value_counts()
temp_data = before_2016_data[before_2016_data.isnull().sum(axis = 1) < 2]
temp_data.isnull().sum(axis = 1)
data_2 = data_2[data_2["Country Name"].isin(olemasolevad_riigid)] #Valime samad riigid mida kasutame esimeses andmestikus


1       2016
2       2016
5       2016
8       2016
9       2016
        ... 
2091    2010
2093    2010
2097    2010
2102    2010
2103    2010
Name: Year, Length: 992, dtype: int64